# HSC Re-Run: Making Forced Photometry Light Curves from Scratch

<br>Owner: **Justin Myles** ([@jtmyles](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@jtmyles))
<br>Last Verified to Run: **N/A -- in development**

This project addresses issue [#63: HSC Re-run](https://github.com/LSSTScienceCollaborations/StackClub/issues/63)

This notebook demonstrates the [LSST Science Piplines data processing tutorial](https://pipelines.lsst.io/) with emphasis on how a given [obs package](https://github.com/lsst/obs_base) works under the hood of the command line tasks. It makes use of the [obs_subaru](https://github.com/lsst/obs_subaru) package to measure a forced photometry light curve for a small patch in the HSC sky in the [ci_hsc](https://github.com/lsst/ci_hsc/) repository. 

### Learning Objectives:
After working through and studying this notebook you should be able to understand how to use the DRP pipeline from image visualization through to a forced photometry light curve. Specific learning objectives include: 
   1. [configure command-line tasks](https://pipelines.lsst.io/v/w-2018-12/modules/lsst.pipe.base/command-line-task-config-howto.html) for your science case
   2. TODO
   
Other techniques that are demonstrated, but not empasized, in this notebook are
   1. Use the `butler` to fetch data
   2. Visualize data with the LSST Stack
   3. TODO

### Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.


## Set Up

In [ ]:
import os
import sys
import eups.setupcmd

## Part I: Interacting with data. Introduction to the Butler
https://pipelines.lsst.io/getting-started/data-setup.html

Part I runs the following command-line tasks
```
eups list lsst_distrib
setup -j -r /home/jmyles/repositories/ci_hsc
echo "lsst.obs.hsc.HscMapper" > /home/jmyles/DATA/_mapper
ingestImages.py /home/jmyles/DATA /home/jmyles/repositories/ci_hsc/raw/*.fits --mode=link
ln -s /home/jmyles/repositories/ci_hsc/CALIB/ /home/jmyles/DATA/CALIB
installTransmissionCurves.py /home/jmyles/DATA
mkdir -p /home/jmyles/DATA/ref_cats
ln -s /home/jmyles/repositories/ci_hsc/ps1_pv3_3pi_20170110 /home/jmyles/DATA/ref_cats/ps1_pv3_3pi_20170110
```

In [ ]:
!eups list lsst_distrib
datarepo = "/home/jmyles/repositories/ci_hsc/"
datadir = "/home/jmyles/DATA/"
os.system("mkdir -p {}".format(datadir))

In [ ]:
#!setup -j -r /home/jmyles/repositories/ci_hsc

setup = eups.setupcmd.EupsSetup(["-j","-r", datarepo])
status = setup.run()
print('setup exited with status {}'.format(status))

A Butler needs a *mapper* file "to find and organize data in a format specific to each camera." We write this file to the data repository so that any instantiated Butler object knows which mapper to use.

In [ ]:
with open(datadir + "_mapper", "w") as f:
    f.write("lsst.obs.hsc.HscMapper")

In [ ]:
# ingest script
!ingestImages.py /home/jmyles/DATA /home/jmyles/repositories/ci_hsc/raw/*.fits --mode=link

In [ ]:
#!installTransmissionCurves.py /home/jmyles/DATA

from lsst.obs.hsc import makeTransmissionCurves, HscMapper
from lsst.daf.persistence import Butler

butler = Butler(outputs={'root': datadir, 'mode': 'rw', 'mapper': HscMapper})

for start, nested in makeTransmissionCurves.getFilterTransmission().items():
    for name, curve in nested.items():
        if curve is not None:
            butler.put(curve, "transmission_filter", filter=name)
for start, nested in makeTransmissionCurves.getSensorTransmission().items():
    for ccd, curve in nested.items():
        if curve is not None:
            butler.put(curve, "transmission_sensor", ccd=ccd)
for start, curve in makeTransmissionCurves.getOpticsTransmission().items():
    if curve is not None:
        butler.put(curve, "transmission_optics")
for start, curve in makeTransmissionCurves.getAtmosphereTransmission().items():
    if curve is not None:
        butler.put(curve, "transmission_atmosphere")

In [ ]:
# ingest calibration images into Butler repo
os.system("ln -s {} {}".format(datarepo + "CALIB/", datadir + "CALIB"))

# ingest reference catalog into Butler repo
os.system("mkdir -p {}".format(datadir + "ref_cats"))
os.system("ln -s {}ps1_pv3_3pi_20170110 {}ref_cats/ps1_pv3_3pi_20170110".format(datarepo, datadir))

# Part 2: Calibrating single frames
https://pipelines.lsst.io/getting-started/processccd.html

In [ ]:
"""
# review what data will be processed
!processCcd.py DATA --rerun processCcdOutputs --id --show data
# id allows you to select data by data ID
# unspecified id selects all raw data
# example IDs: raw, filter, visit, ccd, field
# show data turns on dry-run mode
"""

In [ ]:
#!which processCcd.py

In [ ]:
"""
!processCcd.py DATA --rerun processCcdOutputs --id
# all cl tasks write output datasets to a Butler repo
# --rerun configured to write to processCcdOutputs
# other option is --output
"""

# Part 3: Displaying exposures and source tables output by processCcd.py
https://pipelines.lsst.io/getting-started/display.html

This part of the tutorial is omitted for now.

# Part 4: Coadding images
https://pipelines.lsst.io/getting-started/coaddition.html

* A sky map is a tiling of the celestial sphere. It is composed of one or more tracts.
* A tract is composed of one or more overlapping patches. Each tract has a WCS.

In [ ]:
"""# make a discrete sky map that covers the exposures that have already been processed
!makeDiscreteSkyMap.py DATA --id --rerun processCcdOutputs:coadd --config skyMap.projection="TAN"

# the configuration field specifies the WCS Projection
# one of the FITS WCS projection codes, such as:
#           - STG: stereographic projection
#           - MOL: Molleweide's projection
#           - TAN: tangent-plane projection
"""